In [ ]:
!pip install langchain langchain_huggingface

In [ ]:
import os

In [ ]:
from google.colab import userdata
hf = userdata.get('hf1')

In [ ]:
os.environ["HF_TOKEN"]=hf

In [ ]:
import langchain
import langchain_core
from langchain_huggingface import HuggingFaceEndpoint,ChatHuggingFace

Note: The meta-llama/Llama-3.2-3B-Instruct model is used here for practice and demonstration.
Although it supports with_structured_output(), it is intentionally used to understand parser.get_format_instructions() and output parsing.

In real-world scenarios, many open-source LLMs require output parsers to convert unstructured text into structured data.

In [ ]:
# Type 1 : Comma Separated Output Parser

In [ ]:
# step 1 : Access the LLM
model = HuggingFaceEndpoint(repo_id ="meta-llama/Llama-3.2-3B-Instruct")

In [ ]:
cmodel = ChatHuggingFace(llm=model)

In [ ]:
from typing import TypedDict

In [ ]:
class DF(TypedDict):
  df:str

In [ ]:
cmodel.with_structured_output(DF)

RunnableBinding(bound=ChatHuggingFace(llm=HuggingFaceEndpoint(repo_id='meta-llama/Llama-3.2-3B-Instruct', stop_sequences=[], server_kwargs={}, model_kwargs={}, model='meta-llama/Llama-3.2-3B-Instruct', client=<InferenceClient(model='meta-llama/Llama-3.2-3B-Instruct', timeout=120)>, async_client=<InferenceClient(model='meta-llama/Llama-3.2-3B-Instruct', timeout=120)>), model_id='meta-llama/Llama-3.2-3B-Instruct', temperature=0.8, top_p=0.95, max_tokens=512, model_kwargs={}), kwargs={'tools': [{'type': 'function', 'function': {'name': 'DF', 'description': "dict() -> new empty dictionary\ndict(mapping) -> new dictionary initialized from a mapping object's\n    (key, value) pairs\ndict(iterable) -> new dictionary initialized as if via:\n    d = {}\n    for k, v in iterable:\n        d[k] = v\ndict(**kwargs) -> new dictionary initialized with the name=value pairs\n    in the keyword argument list.  For example:  dict(one=1, two=2)", 'parameters': {'type': 'object', 'properties': {'df': {'ty

In [ ]:
# Type 1 Comma separted Output parser

In [ ]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

In [ ]:
parser = CommaSeparatedListOutputParser()

In [ ]:
parser.get_format_instructions()
# It will return default prompt which is created by CommaSeparatedListOutputParser class.
# And used by us as a system prompt.
# This instraction will be acting as a guidance to the system and to the LLM to generate the output in Comma Separated value.
# And it is constant prompt (Default prompt)

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [ ]:
from langchain_core.prompts import ChatPromptTemplate,HumanMessagePromptTemplate,SystemMessagePromptTemplate

In [ ]:
cpt = ChatPromptTemplate.from_messages([HumanMessagePromptTemplate.from_template("Tell me the pros of {topic}")])

In [ ]:
final_prompt = cpt.format(topic ="Deep learning")

In [ ]:
final_prompt

'Human: Tell me the pros of Deep learning'

In [ ]:
res = cmodel.invoke(final_prompt)

In [ ]:
res.content

'Deep learning is a subfield of machine learning that has revolutionized various fields, including computer vision, natural language processing, and speech recognition. Here are some pros of deep learning:\n\n1. **High Accuracy**: Deep learning models can achieve state-of-the-art performance on a wide range of tasks, including image recognition, speech recognition, and natural language processing.\n\n2. **Flexibility**: Deep learning models can be applied to a variety of tasks and domains, including computer vision, natural language processing, speech recognition, and more.\n\n3. **Automatic Feature Learning**: Deep learning models can automatically learn features from raw data, eliminating the need for manual feature engineering.\n\n4. **Handling High-Dimensional Data**: Deep learning models can handle high-dimensional data, including images, videos, and audio files.\n\n5. **Robustness to Noise**: Deep learning models can learn to be robust to various types of noise, including missing

In [ ]:
# step 2: Create the parser(CSOP)
output = parser.parse(res.content) #parse(text)

# Use the parser function() to convert the input to comma separated output parser.

In [ ]:
output #output is not correct so to improve it we will create a system prompt

['Deep learning is a subfield of machine learning that has revolutionized various fields',
 'including computer vision',
 'natural language processing',
 'and speech recognition. Here are some pros of deep learning:',
 '1. **High Accuracy**: Deep learning models can achieve state-of-the-art performance on a wide range of tasks',
 'including image recognition',
 'speech recognition',
 'and natural language processing.',
 '2. **Flexibility**: Deep learning models can be applied to a variety of tasks and domains',
 'including computer vision',
 'natural language processing',
 'speech recognition',
 'and more.',
 '3. **Automatic Feature Learning**: Deep learning models can automatically learn features from raw data',
 'eliminating the need for manual feature engineering.',
 '4. **Handling High-Dimensional Data**: Deep learning models can handle high-dimensional data',
 'including images',
 'videos',
 'and audio files.',
 '5. **Robustness to Noise**: Deep learning models can learn to be rob

In [ ]:
cpt = ChatPromptTemplate.from_messages([SystemMessagePromptTemplate.from_template("You are a good assistant who will try to give the output in the below format and do not add explaintation or extra text:{format}"),
                                        HumanMessagePromptTemplate.from_template("Tell me the pros of {topic}")])

In [ ]:
prompt = cpt.format(topic="Deep Learning",format=parser.get_format_instructions())


In [ ]:
print(prompt)

System: You are a good assistant who will try to give the output in the below format and do not add explaintation or extra text:Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`
Human: Tell me the pros of Deep Learning


In [ ]:
res =cmodel.invoke(prompt) # step 3

In [ ]:
res.content

'Efficient feature learning, Ability to handle complex data, Improved model interpretability, Robustness to noise and outliers, Ability to learn from large datasets, Flexible and modular architecture.'

In [ ]:
final_out = parser.parse(res.content) #step 4

In [ ]:
final_out

['Efficient feature learning',
 'Ability to handle complex data',
 'Improved model interpretability',
 'Robustness to noise and outliers',
 'Ability to learn from large datasets',
 'Flexible and modular architecture.']

In [ ]:
# Type 2 : String Output parser

In [ ]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser ,StrOutputParser

In [ ]:
# step 1 : Access the model
model = HuggingFaceEndpoint(repo_id="meta-llama/Llama-3.2-3B-Instruct")

In [ ]:
cmodel =ChatHuggingFace(llm=model)


In [ ]:
string_parser = StrOutputParser()

In [ ]:
out = cmodel.invoke("Tell me about agentic AI")

In [ ]:
out

AIMessage(content='Agentic AI refers to a type of artificial intelligence that has the ability to make decisions, act autonomously, and interact with its environment in a purposeful and goal-oriented manner. The term "agentic" comes from the concept of agency, which refers to the capacity of an entity to act and make decisions independently.\n\nAgentic AI systems have several key characteristics:\n\n1. **Autonomy**: They are capable of operating independently, without direct human intervention.\n2. **Intentionality**: They have goals and reasons for acting, and can reason about their own behavior.\n3. **Action**: They can take actions in the world, such as moving, perceiving, or manipulating objects.\n4. **Goal-directed behavior**: They are capable of pursuing goals and tasks, and can adapt to changing circumstances.\n\nAgentic AI systems are often contrasted with traditional AI systems, which are more rule-based and rely on pre-programmed rules and decision-making algorithms. Agentic 

In [ ]:
string_parser.parse(out.content)

'Agentic AI refers to a type of artificial intelligence that has the ability to make decisions, act autonomously, and interact with its environment in a purposeful and goal-oriented manner. The term "agentic" comes from the concept of agency, which refers to the capacity of an entity to act and make decisions independently.\n\nAgentic AI systems have several key characteristics:\n\n1. **Autonomy**: They are capable of operating independently, without direct human intervention.\n2. **Intentionality**: They have goals and reasons for acting, and can reason about their own behavior.\n3. **Action**: They can take actions in the world, such as moving, perceiving, or manipulating objects.\n4. **Goal-directed behavior**: They are capable of pursuing goals and tasks, and can adapt to changing circumstances.\n\nAgentic AI systems are often contrasted with traditional AI systems, which are more rule-based and rely on pre-programmed rules and decision-making algorithms. Agentic AI systems, on the

In [ ]:
# Type 3 -  Json OutputParser

In [ ]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser, JsonOutputParser

In [ ]:
jsp = JsonOutputParser()

# Here if we don't give a data format, it is going to use the default data format.
# Default data format: Which will be having n numbers of keys and values and they will be automatically will be decided by LLM when we are going to give a prompt.


In [ ]:
jsp.get_format_instructions()

'Return a JSON object.'

In [ ]:
cpt2 = ChatPromptTemplate.from_messages([SystemMessagePromptTemplate.from_template("You are a good assistant who will extract the name, age and gender of the person try to give the output in the below format and do not add explaintation or extra text:{format}"),
                                        HumanMessagePromptTemplate.from_template(" MS Dhoni is a great player aged 47 and he is a male")])

In [ ]:
final_pro = cpt2.format(format=jsp.get_format_instructions()) #Output based in instraction

In [ ]:
out_json = cmodel.invoke(final_pro)

In [ ]:
out_json.content

'{"name": "MS Dhoni", "age": 47, "gender": "male"}'

In [ ]:
jsp.parse(out_json.content)

{'name': 'MS Dhoni', 'age': 47, 'gender': 'male'}

In [ ]:
# if we not use output parser just use human and system it will give us string as an output

In [ ]:
cpt1 = ChatPromptTemplate.from_messages([SystemMessagePromptTemplate.from_template("You are a good assistant who will extract the name, age and gender of the person"),
                                        HumanMessagePromptTemplate.from_template(" MS Dhoni is a great player aged 47 and he is a male")])

In [ ]:
prompt =cpt1.format()

In [ ]:
ou1 = cmodel.invoke(prompt)

In [ ]:
ou1.content #it can able to extract infomation but not in JSON format just a simple string only.

'Based on the information provided, I can extract the following details:\n\n- Name: MS Dhoni\n- Age: 47\n- Gender: Male'

In [ ]:
jsp.parse(ou1.content)

# Because this output (ou1.content) whenever we have given it is not aligned to convert into JSON so that is the reason we got error.

OutputParserException: Invalid json output: Based on the information provided, I can extract the following details:

- Name: MS Dhoni
- Age: 47
- Gender: Male
For troubleshooting, visit: https://docs.langchain.com/oss/python/langchain/errors/OUTPUT_PARSING_FAILURE 